In [1]:
import sys
#sys.path.insert(0,'./src/')
#simulator_path = "../covid_de/sim/" ##change simulatio path here 
sys.path.insert(0,'src/')
sys.path.insert(0, '../sib/')
simulator_path = "../simulator/sim/" ##change simulatio path here 
sys.path.insert(0, '../simulator/sim/lib/') # we need distributions.py
sys.path.insert(0,simulator_path)
sys.path.insert(0, "../epidemic_mitigation/src/") # correct rankers

from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json, log, logging, loop_mtpp
from importlib import reload
import imp
from math import exp
import pickle
#sys.path.insert(0,'./src/loop_ranker')
from lib.mobilitysim import MobilitySimulator
import loop_mtpp
from mtpp_utils import contacts_cg
from distributions import CovidDistributions



#logging
data_path = '../simulator/sim/lib/mobility/'

output_dir = "output_Tubingen_pop1_site1/"
fold_out = Path(output_dir)
if not fold_out.exists():
    fold_out.mkdir(parents=True)

reload(log)
logger = log.setup_logger()

Log setup complete


### Mobility simulation

In [2]:
beta = 0.05 # from paper
country = 'GER'
distr = CovidDistributions(country)
#with open(data_path + 'Isle_of_Wight_settings_pop10_site5.pk', 'rb') as fp:
with open(data_path + 'Tubingen_settings_pop1_site1.pk', 'rb') as fp:
    mob_kwargs = pickle.load(fp)

mob_kwargs["delta"] = 0.2554120904376099
T = 20
seed_mob = 2
seed = 1
t_unit = 24
t_res = 0.25 # drop contacts with a duration < t_res (in hours)
max_time = T * t_unit  
mob = MobilitySimulator(**mob_kwargs)
mob.verbose = True
out = mob.simulate(max_time=max_time, seed=seed_mob)
N = mob.num_people 
#print(N)

Simulate mobility for 480.00 time units... 

In [3]:
#contacts_df = pd.DataFrame(contacts_cg(mob, t_res, t_unit, 0, np.inf, first_filter = False),columns = ['i','j','t','deltat'])

In [4]:
#contacts_df = contacts_df.sort_values(by=["t","i","j"])

## Set testing and quarantine rules

In [5]:
#n_indiv=np.ceil(mob_kwargs['num_people_unscaled']/mob_kwargs['downsample_pop'])
n_seeds = {'expo': 3, 'iasy':4,'ipre':5} # select initial infected seeds
num_test_random = 0 #number of random tests per day
fraction_sym_obs = 0.5 #fraction of Symptomatic tested positive
initial_steps = 5 #starting time of intervention
delta_days = 1 # intervention every delta_days days
#assert initial_steps % delta_days == 0
test_HH = False
quarantine_HH = True
adoption_fraction = 1.0
probability_th = 0.00
adapt_th = True

## Set the inference algorithm class

In [6]:
import sib,  scipy

from rankers import sib_rank, dotd_rank, mean_field_rank, tracing_rank
from tqdm.notebook import tqdm
from scipy.stats import gamma
sib.set_num_threads(6)

import matplotlib.pyplot as plt

mu = 1/12
prob_seed = 1/N
prob_sus = 0.5
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-6
fp_rate = 0.0
fn_rate = 0.0

rankers = {}
#rankers["dotd"] = dotd_rank.DotdRanker()

"""
rankers["MF"] = mean_field_rank.MeanFieldRanker(
                tau = 5,
                delta = 10,
                mu = mu,
                lamb = 1.0
                )
                

rankers["tracing"] = tracing_rank.TracingRanker(
                 tau=5,
                 lamb=1.0
)


rankers["greedy"] = greedy_rank.GreedyRanker(
                 include_S = True,
                tau=10)



rankers["winbp20prob0"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Uniform(1.0),
                                 prob_r = sib.Exponential(mu=mu),
                                 #prob_i = sib.PriorDiscrete(list(0.25 * abm_utils.gamma_pdf_array(T+1,6,2.5))),
                                 #prob_r = sib.PriorDiscrete(list(scipy.stats.gamma.sf(range(T+1), 10., scale=1.7452974337097158))),
                                 #prob_i = sib.Uniform(0.014),
                                 #prob_r = sib.Gamma(k = 10, mu = 1/1.7452974337097158),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 30,
                 maxit1 = 30,
                 damp0 = 0.5,
                 damp1 = 0.9,
                 tol = 1e-3,
                 memory_decay = 1e-2,
                 window_length = 20
)


rankers["no_intervention"] = None
"""
k_rec_gamma = 62.484380808876004
scale_rec_gamma = 0.2992112296058585
t0 = distr.incubation_mean_of_lognormal - distr.median_infectious_without_symptom
alpha = 2.0
if probability_th:
        tau = 0
else:
        tau =  7

rankers["sib"] = sib_rank.SibRanker(
                 params = sib.Params(
                                 #prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),
                                 #prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),
                                 prob_i = sib.PiecewiseLinear(sib.RealParams(list(scipy.special.expit(alpha*(range(T+1) -t0*np.ones(T+1)))))), 
                                 prob_r = sib.PiecewiseLinear(sib.RealParams(list(scipy.stats.gamma.sf(range(T+1), k_rec_gamma, scale=scale_rec_gamma)))),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=tau
)

"""
rankers["winbp21_tau0"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),
                                 prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=0
)


rankers["winbp21_uninf"] = winbp_prob0_rank.WinBPProb0Ranker(
                 params = sib.Params(
                                 prob_i = sib.Uniform(1.0), 
                                 prob_r = sib.Exponential(0.1), 
                                 pseed = pseed,
                                 psus = psus,
                                 fp_rate = fp_rate,
                                 fn_rate = fn_rate,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                 tau=7
)

"""

'\nrankers["winbp21_tau0"] = winbp_prob0_rank.WinBPProb0Ranker(\n                 params = sib.Params(\n                                 prob_i = sib.Cached(sib.Scaled(sib.PDF(sib.Gamma(k=5.76, mu=0.96)), scale=0.25), T+1),\n                                 prob_r = sib.Cached(sib.Gamma(k = 10, mu = 1/1.7452974337097158), T+1),\n                                 pseed = pseed,\n                                 psus = psus,\n                                 fp_rate = fp_rate,\n                                 fn_rate = fn_rate,\n                                 pautoinf = pautoinf),\n                 maxit0 = 20,\n                 maxit1 = 20,\n                 tol = 1e-3,\n                 memory_decay = 1e-5,\n                 window_length = 21,\n                 tau=0\n)\n\n\nrankers["winbp21_uninf"] = winbp_prob0_rank.WinBPProb0Ranker(\n                 params = sib.Params(\n                                 prob_i = sib.Uniform(1.0), \n                                 prob_r = sib.E

In [7]:
mob.delta

0.2554120904376099

In [8]:
ress = {}
for num_test_algo in [300]: #number of test per day by ranking
    for seed in [0]:
        print(num_test_algo, seed)
        for s in list(rankers.keys()):
            data = {"algo":s}
            if s== "no_intervention":
                    res_s = loop_mtpp.free_mtpp(mob,
                    country  = country,
                    beta = beta,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_counts = n_seeds,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                     )
            else:
                    res_s = loop_mtpp.loop_mtpp(mob,
                    rankers[s],
                    country  = country,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_steps = initial_steps, 
                    num_test_random = num_test_random,
                    num_test_algo = num_test_algo,
                    fraction_sym_obs = fraction_sym_obs,
                    initial_counts = n_seeds,
                    beta = beta,
                    test_HH = test_HH,
                    quarantine_HH = quarantine_HH,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                    save_every_iter = 1,
                    adoption_fraction = adoption_fraction,
                    probability_th = probability_th,
                    adapt_th = adapt_th,
                    fp_rate = fp_rate,
                    fn_rate = fn_rate
                 )
            ress[s] = res_s    
        del res_s


300 0
Starting with guys:  {'expo': [2529, 14544, 47560], 'iasy': [23291, 62890, 63479, 90427], 'ipre': [19479, 72336, 49207, 59093, 82846]}
t: 24.00 | 480.00 hrs (20 d)

iteration.sib: INFO     time:0



[Reached max time: 24h (1d)]
All raw contacts 984238
Contacts at midnight  32414
Contacts collected by coarse-graining  19656
Contacts in multiple days  0
Coarse-grained contacts  996996
Missing contacts  0
Asymmetric contacts  0


iteration.sib: INFO     adding contacts in the interval [0, 24]
iteration.sib: INFO     number of unique contacts: 964582


iteration.sib: INFO     winBP: (S,I,R): (90538.0, 1.0, 0.0), seeds: 1.0, ll: -62756.4
iteration.sib: INFO     number of tests today: 0


...

iteration.sib: INFO     True  : (S,I,R): (90437.0, 91.0, 11.0)
iteration.sib: INFO     AUR_I : 0.739, prec100: 3, prec(1% of 90459): 0.01, prec5%: 0.00
iteration.sib: INFO     sym: 0, results test algo (S,I,R): (0,0,0), infected test random: 0/0
iteration.sib: INFO     ...quarantining 0 guys -> got 0 infected, 32 free as birds (+10)


t: 479.93 | 480.00 hrs (20 d)

iteration.sib: INFO     time:19


All raw contacts 989390
Contacts at midnight  34474
Contacts collected by coarse-graining  19122
Contacts in multiple days  0
Coarse-grained contacts  1004742
Missing contacts  0
Asymmetric contacts  0


iteration.sib: INFO     adding contacts in the interval [456, 480]
iteration.sib: INFO     number of unique contacts: 967958


In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(1,5,figsize = (25,5))
to_plot = ['I','infected_free','num_quarantined', "q_algo", "n_test_algo"]
for s in ress.keys():    
    for il,l in enumerate(to_plot):
        if s != "no_intervention":
            axs[il].plot(range(T),ress[s][l],label = s)
            if l!= "aurI":
                axs[il].set_yscale ("log") 
                axs[il].vlines(initial_steps,ymin = 1,ymax = 10**4,linestyle = "--",alpha = 0.5, color = "black")
            else:
                axs[il].vlines(initial_steps,ymin = 0,ymax = 1,linestyle = "--",alpha = 0.5, color = "black")
            
            axs[il].set_title(l,fontsize = 12)
    axs[0].legend(loc = 'upper left')
    
#fig.suptitle(f"num test algo {num_test_algo} - adoption_fraction {adoption_fraction}")
fig.show()




In [ ]:
ress['sib']['vecH']